#### Rainfall

In [1]:
import requests
import xarray as xr
import json

In [2]:
# Palau lat/lon
lat = 7.5150
lon = 134.5825

---

List of available sources:

**Current Conditions** 
* [IRI](http://iridl.ldeo.columbia.edu/maproom/Global/Precipitation/?bbox=bb%3A110.10%3A59.56%3A198.92%3A59.80%3Abb) 
    * Monthly anomaly (mm/month)
    
* [NCEI](https://www.ncei.noaa.gov/access/monitoring/usapi-pcp/all/PSW00040309)
    * Daily observations, can be aggregated to weekly or monthly
    * Station data (WEATHER SERVICE OFFICE PALAU AIRAI, ID#PSC00914913, KOROR ID#PSW00040309)
    * Data found [here](https://www.ncei.noaa.gov/access/monitoring/usapi-pcp/all/PSW00040309) is not consistent with the API data...
    * Seems like it's not in real-time, some stations skip days. Currently has data access outages, need to revisit later.
* [NWS](https://forecast.weather.gov/data/obhistory/PTRO.html)
    * No precip data  

**Forecast**  
* [BOM](http://www.bom.gov.au/climate/pacific/outlooks/)
    * Products: Anomaly (difference from average, mm) & Probability of exceeding median (%)
    * Baseline period: 1981-2018
    * Weekly and monthly
    * Weekly updated each Tuesdays/Thursdays, Monthly updated each Thursday  
* [CFSv2](https://www.cpc.ncep.noaa.gov/products/CFSv2/CFSv2_body.html)
    * Monthly anomaly (mm/day)
    * Baseline period: 1991-2020
    * Monthly

**Seasonal outlook**  
* [BOM](http://www.bom.gov.au/climate/pacific/outlooks/)
    * Products: Anomaly (difference from average, mm) & Probability of exceeding median (%)
    * Baseline period: 1981-2018
    * Updated each Thursday
* [CFSv2](https://www.cpc.ncep.noaa.gov/products/CFSv2/CFSv2_body.html)
    * Seasonal anomaly (mm/day)
    * Baseline period: 1991-2020


---

**Current Rainfall Conditions** (Monthly)  
Source: IRI

In [50]:
url = "https://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCEP/.CPC/.CAMS_OPI/.v0208/.anomaly_9120/.prcp/T/%28days%20since%201960-01-01%29streamgridunitconvert/T/differential_mul/T/%28months%20since%201960-01-01%29streamgridunitconvert//units/%28mm/month%29def//long_name/%28Precipitation%20Anomaly%29def/DATA/-500/-450/-400/-350/-300/-250/-200/-150/-100/-50/-25/25/50/100/150/200/250/300/350/400/450/500/VALUES/prcp_anomaly_max500_colors2/Y/%285N%29%2810N%29RANGEEDGES/X/%28130E%29%28140E%29RANGEEDGES/T/%28Nov%202024%29%28Dec%202024%29RANGEEDGES/data.nc"
filename = "iri.monthly.conditions.nc"

response = requests.get(url)
if response.status_code == 200:
    with open(filename, 'wb') as f:
        f.write(response.content)
else:
    print(f"Failed to download file. Status code: {response.status_code}")

ds = xr.open_dataset(filename, decode_times=False, use_cftime=True)
print(ds)

<xarray.Dataset> Size: 96B
Dimensions:  (Y: 2, X: 4, T: 2)
Coordinates:
  * Y        (Y) float32 8B 6.25 8.75
  * X        (X) float32 16B 131.2 133.8 136.2 138.8
  * T        (T) float32 8B 778.5 779.5
Data variables:
    aprod    (T, Y, X) float32 64B ...


In [55]:
ds['aprod'].sel(Y=lat, X=lon, T=779.5,method='nearest').values

array(6.778471, dtype=float32)

Note: Some issues with rendering the time. Need to fix

**Current Rainfall Conditions** (Daily)  
Source: NCEI

In [ ]:
base_url = "https://www.ncei.noaa.gov/access/services/data/v1"

params = {
    "dataset": "daily-summaries",             
    "stations": "PSW00040309",                
    "startDate": "2024-01-06",                
    "endDate": "2024-01-12",                  
    "dataTypes": "PRCP",                      
    "format": "json",                         
    "includeStationName": "true",             
    "units": "metric",               # or standard         
}

# Make the API request
response = requests.get(base_url, params=params)

data = response.json()
print(json.dumps(data, indent=4))

[
    {
        "DATE": "2024-01-07",
        "STATION": "PSW00040309",
        "PRCP": "6.6",
        "NAME": "KOROR, PW PS"
    },
    {
        "DATE": "2024-01-08",
        "STATION": "PSW00040309",
        "PRCP": "1.0",
        "NAME": "KOROR, PW PS"
    },
    {
        "DATE": "2024-01-09",
        "STATION": "PSW00040309",
        "PRCP": "0.8",
        "NAME": "KOROR, PW PS"
    },
    {
        "DATE": "2024-01-10",
        "STATION": "PSW00040309",
        "PRCP": "0.0",
        "NAME": "KOROR, PW PS"
    },
    {
        "DATE": "2024-01-11",
        "STATION": "PSW00040309",
        "PRCP": "2.3",
        "NAME": "KOROR, PW PS"
    },
    {
        "DATE": "2024-01-12",
        "STATION": "PSW00040309",
        "PRCP": "0.0",
        "NAME": "KOROR, PW PS"
    }
]


---

**Weekly rainfall forecast**: Difference from average rainfall forecast (mm)  
Source: [BOM](http://www.bom.gov.au/climate/pacific/outlooks/) <br>
* Baseline period: 1981-2018
* Available products: Weekly, monthly, seasonal forecast
* "One- and three-month outlooks are issued each Thursday, while weekly and fortnightly outlooks
are issued twice a week on Tuesday and Thursday."

In [3]:
url = "https://access-s.clide.cloud/files/global/weekly/data/rain.forecast.anom.weekly.nc"
filename = "rain.forecast.anom.weekly.nc"

response = requests.get(url)
if response.status_code == 200:
    with open(filename, 'wb') as f:
        f.write(response.content)
else:
    print(f"Failed to download file. Status code: {response.status_code}")

rf_weekly_forecast_anomaly_dataset = xr.open_dataset(filename)

In [4]:
#Filter for Palau coordinates
rf_weekly_forecast_anomaly = rf_weekly_forecast_anomaly_dataset['rain'].sel(lat=lat, lon=lon, method='nearest')

# Convert to a Pandas DataFrame
rf_weekly_forecast_anomaly_df = rf_weekly_forecast_anomaly.to_dataframe().reset_index()
print(rf_weekly_forecast_anomaly_df)


                 time         lon  lat       rain
0 2025-01-09 12:00:00  134.583328  7.5  17.108965
1 2025-01-16 12:00:00  134.583328  7.5 -22.067575
2 2025-01-23 12:00:00  134.583328  7.5  14.037240
3 2025-01-30 12:00:00  134.583328  7.5  16.520462
4 2025-02-06 12:00:00  134.583328  7.5  13.158188


---

**Weekly rainfall forecast**: Chance of exceeding the median rainfall (%)  
Source: [BOM](http://www.bom.gov.au/climate/pacific/outlooks/) <br>

In [5]:
url = "https://access-s.clide.cloud/files/global/weekly/data/rain.forecast.median.weekly.nc"
filename = "rain.forecast.median.weekly.nc"

response = requests.get(url)
if response.status_code == 200:
    with open(filename, 'wb') as f:
        f.write(response.content)
else:
    print(f"Failed to download file. Status code: {response.status_code}")

rf_weekly_forecast_probability_dataset = xr.open_dataset(filename)

In [6]:
#Filter for Palau coordinates
rf_forecast_probability = rf_weekly_forecast_probability_dataset['probF'].sel(lat=lat, lon=lon, method='nearest')

# Convert to a Pandas DataFrame
rf_forecast_probability_df = rf_forecast_probability.to_dataframe().reset_index()
print(rf_forecast_probability_df)


                 time         lon  lat      probF
0 2025-01-09 12:00:00  134.583328  7.5  97.979797
1 2025-01-16 12:00:00  134.583328  7.5   8.080808
2 2025-01-23 12:00:00  134.583328  7.5  80.808083
3 2025-01-30 12:00:00  134.583328  7.5  77.777779
4 2025-02-06 12:00:00  134.583328  7.5  81.818184


---

#### Seasonal Rainfall Outlook  

**Seasonal rainfall outlook**: Difference from average (mm), 3-month period  
Source: [BOM](http://www.bom.gov.au/climate/pacific/outlooks/)  

Baseline period: 1981-2018

In [7]:
url = "https://access-s.clide.cloud/files/global/seasonal/data/rain.forecast.anom.seasonal.nc"
filename = "rain.forecast.anom.seasonal.nc"

response = requests.get(url)
if response.status_code == 200:
    with open(filename, 'wb') as f:
        f.write(response.content)
else:
    print(f"Failed to download file. Status code: {response.status_code}")

rf_seasonal_outlook_anomaly_dataset = xr.open_dataset(filename)

In [8]:
rf_seasonal_outlook_anomaly = rf_seasonal_outlook_anomaly_dataset['rain'].sel(lat=lat, lon=lon, method='nearest')

# Convert to a Pandas DataFrame
rf_seasonal_outlook_anomaly_df = rf_seasonal_outlook_anomaly.to_dataframe().reset_index()
print(rf_seasonal_outlook_anomaly_df)


                           time         lon  lat        rain
0 2025-03-16 20:00:56.249999872  134.583328  7.5  185.423203
1 2025-04-16 00:00:00.000000000  134.583328  7.5  140.219925
2 2025-05-16 12:00:00.000000000  134.583328  7.5   55.053329


Notes: Unclear what the time is referring to. <br>
Assuming 2025-03-16 refers to February to April, taking the middle date of the period <br>

---

**Seasonal rainfall outlook**: Chance of exceeding median rainfall (%), 3-month period  
Source: [BOM](http://www.bom.gov.au/climate/pacific/outlooks/)  

In [9]:
url = "https://access-s.clide.cloud/files/global/seasonal/data/rain.forecast.median.seasonal.nc"
filename = "rain.forecast.median.seasonal.nc"

response = requests.get(url)
if response.status_code == 200:
    with open(filename, 'wb') as f:
        f.write(response.content)
else:
    print(f"Failed to download file. Status code: {response.status_code}")

rf_seasonal_outlook_probability_dataset = xr.open_dataset(filename)

In [10]:
rf_seasonal_outlook_probability = rf_seasonal_outlook_probability_dataset['probF'].sel(lat=lat, lon=lon, method='nearest')

# Convert to a Pandas DataFrame
rf_seasonal_outlook_probability_df = rf_seasonal_outlook_probability.to_dataframe().reset_index()
print(rf_seasonal_outlook_probability_df)


                           time         lon  lat      probF
0 2025-03-16 20:00:56.249999872  134.583328  7.5  88.888885
1 2025-04-16 00:00:00.000000000  134.583328  7.5  84.848488
2 2025-05-16 12:00:00.000000000  134.583328  7.5  61.616161


---

**Seasonal rainfall outlook**: Anomaly (mm/day), 3-month period  
Source: [CFSv2](https://www.cpc.ncep.noaa.gov/products/CFSv2/CFSv2_body.html)

In [42]:
url = "https://www.cpc.ncep.noaa.gov/products/CFSv2/dataInd1/glbPrecSea.nc"
filename = "cfs.rainfall.seasonal.nc"

response = requests.get(url)
if response.status_code == 200:
    with open(filename, 'wb') as f:
        f.write(response.content)
else:
    print(f"Failed to download file. Status code: {response.status_code}")

cfs_dataset = xr.open_dataset(filename)

In [43]:
cfs = cfs_dataset['anom'].sel(lat=lat, lon=lon, method='nearest')

# Convert to a Pandas DataFrame
cfs_df = cfs.to_dataframe().reset_index()
print(cfs_df)


   ens       time  lev    lon     lat      anom
0  1.0 2025-02-01  1.0  135.0  7.0866  2.116728
1  1.0 2025-03-01  1.0  135.0  7.0866  0.742680
2  1.0 2025-04-01  1.0  135.0  7.0866  0.131184
3  1.0 2025-05-01  1.0  135.0  7.0866  0.763008
4  1.0 2025-06-01  1.0  135.0  7.0866  1.363344
5  1.0 2025-07-01  1.0  135.0  7.0866  0.919008
